In [1]:
import os 
from pathlib import Path # For fetching the required files
import pandas as pd

In [72]:
DirPpath = Path(os.path.abspath('')).parent # Fetching the current directory path

#ResultsPath  = str(DirPpath.absolute()) + "\OutputFiles\Clusters.xlsx"
#ResultsPath  = str(DirPpath.absolute()) + "\OutputFiles\Clusters2.xlsx"
#ResultsPath  = str(DirPpath.absolute()) + "\OutputFiles\Clusters3.xlsx"
#ResultsPath  = str(DirPpath.absolute()) + "\OutputFiles\Clusters4.xlsx"
ResultsPath  = str(DirPpath.absolute()) + "\OutputFiles\Clusters5.xlsx"
ResultsDf = pd.read_excel(ResultsPath)  

In [ ]:
ResultsDf

### Splitting the clusters based on prompt size limit

In [97]:
cluster_txt = {}

for i in range(1,7):
    
    print(i)
    cluster = ResultsDf[ResultsDf["Cluster"] == i]
    text = ""
    
    j = 0
    cluster_txt[i] = []

    for pledge in cluster["Pledge"]: 
        
        j +=1 

        new_text = text + " Text " + str(j) + ": " + pledge + "\n"

        # if len(new_text) > 3900*4:
        
        #      cluster_txt[i].append(text)
        #      text = " Text " + str(j) + ": " + pledge + "\n"

        # else:
        text = new_text
        
        
        
    cluster_txt[i].append(text)

1
2
3
4
5
6


In [98]:
sum = 0

for cluster in cluster_txt:

    length = len(cluster_txt[cluster])
    print(length)
    sum += length

print(sum)

1
1
1
1
1
1
6


In [102]:
print(cluster_txt[6][0])

 Text 1: Achieve sustainable and flexible solutions for multimodal transport and develop policies to protect natural heritage and biodiversity, respecting the socio-cultural authenticity of host communities.

CNA Turismo e Commercio, over the three-year period (between autumn 2022 and throughout 2023), will organize training seminars for businesses - with the involvement of public and private stakeholders - aimed at the implementation of concrete solutions for the development of good practices for a supply of multimodal transport and protocols for the respect and protection of the natural heritage and biodiversity.
 Text 2: As an organisation representing transport workers, we are keen to ensure that there are realistic, practical and green intermodal transport solutions to support green travel of passengers to/from tourism destinations. The ETF represents workers in all transport modes, and we regularly work together to come up with common intermodal solutions, including under the TEN

### Creating the prompt

In [95]:
for keys in cluster_txt.keys():
    
    for i in range(len(cluster_txt[keys])):

        
        if i == 0: 
            cluster_txt[keys][i] = f"""Do not answer yet. This is just another part of the text I want to send you. Just receive and acknowledge as "Part 1/{len(cluster_txt[keys])} received" and wait for the next part.             
            [START PART 1/{len(cluster_txt[keys])}]            
            Below you can find a set of texts belonging to a common cluster. Based on their content, I want you to tell me what is the topic of this cluster:            
            {cluster_txt[keys][i]}
            [END PART 1/{len(cluster_txt[keys])}]  
            Remember not answering yet. Just acknowledge you received this part with the message "Part 1/{len(cluster_txt[keys])} received" and wait for the next part.
            """

        elif i == (len(cluster_txt[keys])-1):
            cluster_txt[keys][i] = f"""[START PART {len(cluster_txt[keys])}/{len(cluster_txt[keys])}]    
            {cluster_txt[keys][i]}
            [END PART {len(cluster_txt[keys])}/{len(cluster_txt[keys])}]
            ALL PARTS SENT. Now you can continue processing the request.
            """

        else: 
            cluster_txt[keys][i] = f"""Do not answer yet. This is just another part of the text I want to send you. Just receive and acknowledge as "Part {i+1}/{len(cluster_txt[keys])} received" and wait for the next part.  
            
            [START PART {i+1}/{len(cluster_txt[keys])}]
            {cluster_txt[keys][i]}
            [END PART {i+1}/{len(cluster_txt[keys])}]
            Remember not answering yet. Just acknowledge you received this part with the message "Part {i+1}/{len(cluster_txt[keys])} received" and wait for the next part.
            """

    content = f"""The total length of the content that I want to send you is too large to send in only one piece. For sending you that content, I will follow this rule:              
    [START PART 1/{len(cluster_txt[keys])}]  
    this is the content of the part 1 out of {len(cluster_txt[keys])} in total  
    [END PART 1/{len(cluster_txt[keys])}]  
    Then you just answer: "Received part 1/{len(cluster_txt[keys])}"  
    And when I tell you "ALL PARTS SENT", then you can continue processing the data and answering my requests.

    """

    cluster_txt[keys].insert(0, content)

In [96]:
print(cluster_txt[5][1])

Do not answer yet. This is just another part of the text I want to send you. Just receive and acknowledge as "Part 1/1 received" and wait for the next part.             
            [START PART 1/1]            
            Below you can find a set of texts belonging to a common cluster. Based on their content, I want you to tell me what is the topic of this cluster:            
             Text 1: IN2destination actively voluntarily and under contract works with tourism destinations to improve their statistical data and indicators, such as developing sustainable tourism indicators to inform tourism policy decisions. The company also provides training and capacity-building for tourism stakeholders in data collection and analysis. IN2destination's concrete target for 2025 is to assist at least 15 destinations in improving their statistical data and indicators.
 Text 2: IN2destination supports destinations in developing comprehensive tourism strategies that are inclusive, sustainable, an

### API calls to openAI (classification)

In [103]:
import openai
import os

#os.environ["OPENAI"] = "XXXX"
openai.api_key = os.environ["OPENAI"]

ModuleNotFoundError: No module named 'openai'

#### Cluster definitions

Word2vec

In [ ]:
Cluster1 = "the common topic of Cluster 1 is related to sustainable tourism development. The texts emphasize various aspects such as governance, monitoring, innovation, digital transformation, sustainable practices, and stakeholder engagement. The primary goal is to promote a sustainable and responsible approach to tourism, addressing environmental, social, and economic sustainability while enhancing the competitiveness and resilience of the tourism industry."
Cluster2 = "The common topic of Cluster 2 is related to sustainability and green initiatives in the tourism industry. The various texts discuss measures and goals aimed at reducing carbon emissions, promoting eco-friendly practices, sustainable tourism development, and the overall transition towards a more environmentally responsible tourism sector. This includes initiatives such as promoting sustainable accommodation, environmentally friendly transportation, reducing waste and plastic use, and supporting certification programs for sustainable tourism businesses."
Clutser3 = "The common topic of Cluster 3 is related to data-driven decision-making and the use of data for improving various aspects of the tourism industry. The texts discuss the development and implementation of data platforms, monitoring systems, and data analysis tools to better understand visitor behavior, enhance destination management, reduce environmental impacts, and improve overall tourism experiences. This includes initiatives such as creating data lakes, establishing data sharing practices, developing data-driven tourism services, and utilizing big data sources for predicting tourist flows."
Cluster4 = "The common topic of Cluster 4 is related to the development and implementation of digital solutions, platforms, and initiatives aimed at improving the tourism industry. This includes enhancing accessibility, sustainability, inclusivity, and visibility of tourism offerings for various destinations, businesses, and stakeholders. The cluster also covers the use of digital technologies for better communication, data management, marketing, and enhancing visitor experiences. The overarching goal is to support tourism authorities, destinations, and SMEs in adopting digital technologies and practices to create a more efficient, sustainable, and inclusive tourism industry by 2025 and beyond."
Cluster5 = "The common topic of Cluster 5 is related to the development and improvement of the tourism sector, with a focus on sustainability, digitalization, skills development, and inclusivity. The texts mention various initiatives, projects, and goals aimed at enhancing the tourism industry's transition towards a more sustainable, inclusive, and resilient future, involving collaboration between educational institutions, businesses, governments, and other stakeholders."
Cluster6 = "The common topic of Cluster 6 is related to investing in and supporting innovative solutions across various sectors, such as sustainable mobility, circular economy, green transition, tourism, and seamless travel. The focus is on identifying and mentoring entrepreneurs and startups, with the goal of becoming their first investor and assisting in their growth."

BERT

In [ ]:
Cluster1 = "The common topic of Cluster 1 is related to the development and implementation of various strategies, initiatives, and actions aimed at improving the sustainability, accessibility, and competitiveness of the tourism sector in Europe. The texts discuss efforts to promote fair competition, ensure environmental responsibility, enhance digitalization, improve workforce skills, and support innovation in the tourism industry. They also emphasize the importance of collaboration between various stakeholders, such as governments, businesses, educational institutions, and international organizations, to achieve these goals and enhance the overall quality and sustainability of the tourism sector."
Cluster2 = "The common topic of Cluster 2 is related to the development and implementation of various strategies, initiatives, and actions aimed at improving the sustainability, digitalization, accessibility, and competitiveness of the tourism sector in Europe. The texts discuss efforts to promote fair competition, ensure environmental responsibility, enhance digitalization, improve workforce skills, and support innovation in the tourism industry. They also emphasize the importance of collaboration between various stakeholders, such as governments, businesses, educational institutions, and international organizations, to achieve these goals and enhance the overall quality and sustainability of the tourism sector."
Clutser3 = "The common topic of Cluster 3 is centered around sustainability, green transition, and digital transformation in the tourism sector. The texts discuss various initiatives, projects, and commitments aimed at improving environmental, social, and economic aspects of tourism, while also addressing skills development, data-driven approaches, and accessibility. The goal is to create a more sustainable, resilient, and inclusive tourism industry across Europe and beyond."
Cluster4 = "The common topic of Cluster 4 is Sustainable Tourism and Data Management. The texts mainly discuss initiatives, projects, and commitments related to enhancing sustainability in the tourism sector, promoting environmentally friendly practices, reducing waste and carbon footprint, and improving data collection and sharing for informed decision-making. The texts emphasize the importance of collaboration among stakeholders, developing innovative solutions, and utilizing technology and data to better understand and manage tourism while minimizing negative impacts on the environment, economy, and society."
Cluster5 = "The common topic of Cluster 5 is related to the sustainable tourism development. The texts discuss various aspects of this topic, including digitalization, environmental protection, data-driven approaches, green transition, education and training, funding opportunities, and stakeholder collaboration. The focus is on improving tourism practices, reducing the negative impact on the environment, and enhancing the overall sustainability of the tourism sector."
Cluster6 = "The common topic of Cluster 6 is related to sustainable tourism, digitalization, and inclusivity. The texts discuss various initiatives, strategies, projects, and actions aimed at promoting environmentally friendly practices, digital transformation, accessibility, and social responsibility within the tourism industry. Some key areas of focus include sustainable development, workforce training, digital tools, data sharing, and accessibility for people with disabilities."

TourBERT

In [ ]:
Cluster1 = "The common topic of Cluster 1 is related to sustainable tourism, tourism development strategies, and the digitalization and green transition of the tourism industry. The texts discuss various aspects of sustainable tourism, including certifications, data management, national and regional strategies, destination management, capacity building, and education and training initiatives."
Cluster2 = "The common topic of Cluster 2 is focused on the development and implementation of sustainable and accessible tourism practices. This includes initiatives and commitments towards a green transition, digital transformation, and resilience within the tourism industry. The texts discuss various projects, policies, and actions aiming to improve the environmental, social, and economic aspects of the tourism sector across Europe and beyond."
Clutser3 = "The common topic of Cluster 3 is related to sustainable and accessible tourism. The texts discuss various initiatives, projects, and goals aimed at improving the tourism industry by focusing on sustainability, digitalization, accessibility, capacity building, and skill development. These efforts are meant to create a more inclusive, environmentally friendly, and economically viable tourism sector for both local communities and visitors."
Cluster4 = "The common topic of Cluster 4 is related to sustainable tourism development, with a focus on digitalization, accessibility, environmental protection, capacity building, and skill development. The texts discuss various initiatives, projects, and goals aimed at improving the tourism industry by focusing on sustainability, green transition, community involvement, and the use of digital technologies. These efforts are meant to create a more inclusive, environmentally friendly, and economically viable tourism sector for both local communities and visitors."
Cluster5 = "The common topic of Cluster 5 is related to sustainable and inclusive tourism development, with a focus on digitalization, accessibility, environmental protection, capacity building, and skill development. The texts discuss various initiatives, projects, and goals aimed at improving the tourism industry by focusing on sustainability, green transition, community involvement, and the use of digital technologies. These efforts are meant to create a more inclusive, environmentally friendly, and economically viable tourism sector for both local communities and visitors."
Cluster6 = "The common topic of Cluster 6 is related to promoting sustainable, responsible, and inclusive tourism practices through various initiatives, such as improving data-driven approaches, digitalization, intermodal transportation, circular economy models, and stakeholder engagement. The texts highlight strategies, projects, and targets aimed at enhancing the tourism industry's sustainability, accessibility, and resilience, while also emphasizing the importance of collaboration between public, private, and civil society sectors."

Fine-tuned BERT

In [ ]:
Cluster1 = "The common topic of Cluster 1 is related to the tourism industry, with a particular focus on sustainability, accessibility, digitalization, skills development, and inclusivity. The texts discuss various initiatives, projects, and commitments by different organizations, regions, and stakeholders to improve and transform the tourism sector in alignment with the Sustainable Development Goals, digital transformation, and providing equal opportunities for all."
Cluster2 = "The common topic of Cluster 2 revolves around sustainable and accessible tourism. The texts discuss various initiatives, projects, and commitments aimed at reducing the environmental impact of tourism, enhancing accessibility for people with disabilities, promoting social and economic sustainability, and fostering digital and green transformations in the tourism sector. The texts also outline goals, actions, and measurable targets to support the transition towards more sustainable and inclusive tourism practices, both in the short and long term."
Clutser3 = "The common topic of Cluster 3 is related to sustainable tourism, focusing on various aspects such as climate neutrality, CO2 emissions reduction, environmental and social indicators, accessibility, digitalization, greenways, and local community engagement. The texts discuss different projects, initiatives, and commitments aimed at promoting sustainability in the tourism sector across various regions and countries."
Cluster4 = "The common topic of Cluster 4 is related to sustainable and inclusive tourism, with a focus on various aspects such as environmental responsibility, digital transformation, accessibility, education, green transition, and social responsibility. The texts discuss different projects, initiatives, and commitments aimed at promoting sustainability, inclusivity, and accessibility in the tourism sector across various regions and countries."
Cluster5 = "The common topic of Cluster 5 is related to the development, implementation, and monitoring of strategies, policies, and initiatives in the tourism sector. These texts cover a wide range of subjects, including sustainable tourism, digital transformation, data management, education and training, accessibility, green transition, and governance. The focus is on enhancing the tourism experience for visitors, improving the sector's environmental and social impact, and ensuring the industry's long-term viability and growth."
Cluster6 = "The common topic of Cluster 6 is related to sustainable tourism development, focusing on various aspects such as digitalization, education and training, environmental sustainability, data-driven approaches, and destination management. These texts discuss initiatives, projects, and goals aimed at improving the tourism sector in terms of sustainability, accessibility, and inclusivity. The focus is on providing support and resources for stakeholders, such as SMEs and destinations, to adopt more sustainable practices, leverage digital technologies, and enhance the overall tourism experience for visitors and residents alike."

Roberta

In [ ]:
Cluster1 = "The common topic of Cluster 1 us related to sustainable tourism. These pledges include initiatives to promote sustainable tourism, improve accessibility, digitize tourism services, and protect marine life. The ultimate goal is to create a more sustainable and responsible tourism industry."
Cluster2 = "The pledges contained in Cluster 2 outline various actions and targets related to sustainable tourism, such as developing policies to protect natural heritage and biodiversity, promoting sustainable tourism indicators, and reducing the negative impacts of tourism on resident communities. Additionally, the pledges mention the need for sustainable and flexible solutions for multimodal transport and the implementation of concrete solutions for the development of good practices for a supply of multimodal transport."
Cluster3 = "The pledges belonging to Cluster 3 aim to promote sustainable tourism practices, such as the development of sustainable services, the reduction of greenhouse gas emissions, the use of renewable energy sources, the introduction of zero-emission vehicles, the implementation of a system of indicators for measuring sustainability at the local level, and the establishment of collaborative destination governance mechanisms. The ultimate goal of these pledges is to enhance the quality of life, improve air quality, and ensure a sustainable and innovative recovery and development of tourism."
Cluster4 = "The pledges belonging to Cluster 4 cover a wide range of topics, such as digitalization, circular economy, sustainability, and workforce development. The pledges aim to promote sustainable tourism practices, reduce the negative impact of tourism on the environment, and improve the quality of life for local communities."
Cluster5 = "The common topic of CLuster 5 is tourism and destination management through digital platforms and data-driven strategies. The cluster discusses various aspects such as the development and implementation of digital platforms, collaboration between stakeholders in destinations, tourism governance, sustainability goals, tourism data sharing, and the use of technology for offering seamless experiences to travelers. Additionally, it touches upon training initiatives in the hospitality industry related to online marketing and digitalization." 
Cluster6 = "The common topic of Cluster 6 is sustainable tourism and destination management. The cluster discusses various initiatives and commitments aimed at promoting sustainability in the tourism sector. This includes participation in Smart Destination Networks, knowledge sharing events on tourism education with a focus on circular economy, measures to reduce carbon footprint, setting targets for green labels, implementing digital transformation programs for tourism SMEs, and specific training programs for employee development. Additionally, there is a focus on providing support and becoming a one-stop-shop for agricultural estates engaged in hospitality services, further emphasizing the sustainable and responsible aspects of tourism."

RobertaF

In [ ]:
Cluster1 = "The common topic of this cluster is the commitment to digitalization and data-driven approaches to improve the tourism industry. The pledges include the development of digital platforms and tools, the collection and sharing of tourism data, the promotion of digital readiness among SMEs, the use of innovative technologies, and the provision of digital training and support. The ultimate goal is to improve the competitiveness and sustainability of the tourism industry through the use of digital technologies and data-driven solutions"
Cluster2 = "The pledges of Cluster 2 include commitments to develop sustainable tourism products, measure and track GHG emissions, report on progress, support the digitalization of the tourism sector, develop new skills profiles, and enhance the competitiveness and sustainability of destinations. The pledges also aim to promote the use of service design and digitization of services in tourism companies, increase the use of renewable energy sources, and stimulate the development of zero-emission vehicles and infrastructure. Additionally, the pledges aim to enhance the quality of life, improve air quality, and increase the resilience of critical infrastructure."
Cluster3 = "The common topic of Cluster 3 is sustainable tourism development, digitalization, and the use of technology in the tourism sector. The cluster discusses various initiatives related to sustainability, including the development of monitoring schemes for carbon footprints, establishing knowledge-sharing events on circular economy in tourism, implementing measures to reduce environmental impact, setting targets for green labels, and developing strategies for sustainable tourism at regional, national, and European levels. Additionally, there is a focus on digital transformation, providing training and support to tourism SMEs, and utilizing technology for innovation and sustainability in the tourism industry. The texts also touch upon collaboration, networking, and partnerships to promote sustainable practices and share best practices in the tourism sector."
Cluster4 = "The pledges of Cluster 4 aim to promote sustainable practices in the tourism industry, such as reducing carbon footprint, promoting circularity, and increasing the use of renewable energy. The cluster includes various actions, such as providing sustainable tourism training, developing sustainable tourism guidelines, promoting sustainable entrepreneurship, and creating sustainable tourism networks. The ultimate goal of these pledges is to create a more sustainable tourism industry that benefits both the environment and local communities."
Cluster5 = "The common topic across Cluster 5 is sustainable tourism development, destination management, and capacity building. The cluster revolves around efforts to enhance the sustainability of tourism through various initiatives. Key themes include: Destination Development: Texts highlight support for destinations in improving statistical data, developing comprehensive tourism strategies, and advocating for collaborative governance models. Sustainable Mobility: Some texts focus on promoting sustainable mobility options for tourism, such as public transportation, cycling, and walking. Circular Economy: There is a focus on supporting circular economy models in tourism, including reducing waste, using renewable energy, and supporting local businesses. Carbon Footprint Reduction: Actions are taken to reduce the carbon footprint in business operations and implement measures to validate and evaluate sustainability efforts. Education and Capacity Building: Initiatives include knowledge-sharing events on tourism education, circular economy, and sustainable practices. There is also an emphasis on training programs, both for employees and tourism stakeholders, to develop necessary skills. Investment in Sustainable Solutions: Efforts are made to identify, mentor, and invest in companies and innovations that contribute to sustainable mobility, circular economy, and green transition in tourism. Digitalization: The cluster addresses the role of digital technologies in promoting sustainability, with a focus on digital solutions, digital marketing, and online booking systems. Support for SMEs: Actions involve providing technical and financial support to tourism SMEs for green transition, digital transformation, and overall sustainability. Cooperation and Networking: Collaboration is emphasized, including engagement with Smart Destination Networks, cooperation projects, and partnerships with educational organizations. Accessing Funding Opportunities: There is a commitment to assisting tourism actors in accessing funding opportunities, disseminating information about funding programs, and developing proposals. Overall, the cluster reflects a comprehensive approach to fostering sustainability in the tourism sector, encompassing various dimensions such as environmental conservation, community engagement, circular economy practices, and the integration of digital technologies."
Cluster6 = "The pledges of Cluster 6 cover a wide range of topics, including the development of sustainable tourism strategies, the implementation of sustainable tourism policies, the promotion of sustainable tourism offers, the adoption of sustainable tourism practices, the measurement and reduction of the ecological footprint of tourism, the protection of natural heritage and biodiversity, and the facilitation of intermodal transport solutions. The ultimate goal of these pledges is to achieve a more sustainable, resilient, and digital tourism industry."

#### Classification

In [ ]:
#Note: The openai-python library support for Azure OpenAI is in preview.
      #Note: This code sample requires OpenAI Python library version 0.28.1 or lower.
import os
import re
import openai

openai.api_type = "azure"
openai.api_base = "https://eastusazuredemo.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
#openai.api_key = os.getenv("OPENAI_API_KEY")

prediction = []

for texts in ResultsDf.iloc[300:,:]["Pledge"]:
  message_text = [{"role":"system","content":f"You are an AI assistant that helps people find information.You will be provided with different textual pledges. Classify each pledge into Cluster 1, Cluster 2, Cluster 3, Cluster 4, Cluster 5, or Cluster 6. {Cluster1} {Cluster2} {Clutser3} {Cluster4} {Cluster5} {Cluster6}"}
                  ,{"role":"user","content":f"Here is the pledge to classify: {texts}. Only provide the Cluster number as output"}]

  completion = openai.ChatCompletion.create(
    engine="basetest",
    messages = message_text,
    temperature=0.7,
    max_tokens=800,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
  )

  prediction.append(re.sub(r'Cluster\s([0-9])', r'\1' ,completion.choices[0]["message"]["content"]))

In [ ]:
for i in range(len(prediction)):
    
    try:
        prediction[i] = re.sub(r'[^\d]+', '', prediction[i])
        prediction[i] = int(prediction[i])
    except:
        next

In [ ]:
total = 0
count = 0

for cluster, pred in zip(prediction, ResultsDf.iloc[:,:]["Cluster"]):

    total +=1 

    if cluster == pred:
        
        count += 1

In [ ]:
count/total

In [ ]:
df = pd.DataFrame.from_dict({"Prediction": prediction, "Cluster": ResultsDf.iloc[:,:]["Cluster"]})

pd.crosstab(df["Cluster"], df["Prediction"])

In [ ]:
from sklearn.metrics import f1_score

f1_score(df["Prediction"], df["Cluster"], average="weighted")

In [ ]:
predictionTour #= prediction.copy()

In [ ]:
predictionFBERT #= prediction.copy()

In [ ]:
results = pd.DataFrame({"TourBERT": predictionTour, "Fine-tuned BERT": predictionFBERT})
results.to_csv('results.csv')

Word2vec: 48%, 0.47 F1
BERT: 22%, 0.25 F1
Tour BERT: 19%, 0.24 F1
Fine-tuned BERT: 29%, 0.34 F1